In [1]:
pip install sklearn-recommender


Note: you may need to restart the kernel to use updated packages.


In [2]:
%matplotlib inline

import pandas
from sklearn.model_selection import train_test_split

import numpy as np
import time
from sklearn.externals import joblib


/Users/devarakondasantosh/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
triplets_file = '10000.txt'
songs_metadata_file = 'song_data.csv'

In [4]:
#importing all the required libraries
import numpy as np
import pandas as pd
#importing both the datasets
songmetadata = pd.read_csv(r'song_data.csv')
#one of the file is a text file hence we import it with pd.read_fwf
#fwf stands for fixed width file
othersongdata = pd.read_fwf(r'10000.txt')


In [5]:
#naming the columns for the othersongdata
othersongdata.columns = ['user_id','song_id','listen_count']

In [6]:
#merging both the datasets and removing duplicates
song_df = pd.merge(othersongdata, songmetadata.drop_duplicates(['song_id']), on="song_id", how="left")
#writing the file in .csv to visualize in Tableau
song_df.to_csv(r'Processed_Data.csv', index = False)

In [7]:
song_df

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Graduation,Kanye West,2007
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,In Between Dreams,Jack Johnson,2005
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N' Roll,Antología Audiovisual,Héroes del Silencio,2007
...,...,...,...,...,...,...,...
1999994,d8bfd4ec88f0f3773a9e022e3c1a0f1d3b7b6a92,SOJEYPO12AAA8C6B0E,2,Ignorance (Album Version),Ignorance,Paramore,0
1999995,d8bfd4ec88f0f3773a9e022e3c1a0f1d3b7b6a92,SOJJYDE12AF729FC16,4,Two Is Better Than One,Love Drunk,Boys Like Girls featuring Taylor Swift,2009
1999996,d8bfd4ec88f0f3773a9e022e3c1a0f1d3b7b6a92,SOJKQSF12A6D4F5EE9,3,What I've Done (Album Version),What I've Done,Linkin Park,2007
1999997,d8bfd4ec88f0f3773a9e022e3c1a0f1d3b7b6a92,SOJUXGA12AC961885C,1,Up,My Worlds,Justin Bieber,2010


# Step 1: Data preprocessing completed

# Step 2: Recommender System 1 - Popular Playlist

In [8]:
import pandas as pd
songmetadata = pd.read_csv(r'song_data.csv')
othersongdata = pd.read_fwf(r'10000.txt')
songmetadata = pd.DataFrame(songmetadata)
othersongdata.columns = ['user_id','song_id','listen_count']
song_df = pd.merge(othersongdata, songmetadata.drop_duplicates(['song_id']), on="song_id", how ="left")
song_grouped = song_df.groupby(['title']).agg({"listen_count":"count"})
grouped_sum = song_grouped['listen_count'].sum()

#calculating the percent share of each song in listen count
song_grouped['percentage'] = song_grouped['listen_count'].div(grouped_sum)*100

#sorting the dataset with respect to listen count
song_grouped = song_grouped.sort_values(['listen_count'],ascending = True)
song_df = song_df['listen_count'].astype(float)
popular = song_grouped.sort_values(by = 'listen_count')

#filtering the top ten songs in the dataset
# popularsongs = popular[9517:9567]
popularsongs = popular
popularsongs = pd.DataFrame(popularsongs.reset_index())
recommendedSongs = popularsongs.sort_values('listen_count', ascending = False)
# popularsongs

In [9]:
recommendedSongs

,title,listen_count,percentage
9566,Sehr kosmisch,8277,0.41385
9565,Undo,7032,0.35160
9564,Dog Days Are Over (Radio Edit),6949,0.34745
9563,You're The One,6729,0.33645
9562,Revelry,6145,0.30725
...,...,...,...
4,Scared,51,0.00255
3,Historia Del Portero,51,0.00255
2,Don´t Leave Me Now,50,0.00250
1,No Creo En El Jamas,48,0.00240


# Step 3: Recommender System 2 - Item based Similarities

In [10]:
givenSongID_From_FrontEnd = 'SOFVZRE12A8C139783'

In [11]:
import numpy as np
import pandas as pd
# songmetadata = pd.read_csv(r'E:\Analytics\song_data.csv')
# othersongdata = pd.read_fwf(r'E:\Analytics\10000.txt')
othersongdata.columns = ['user_id','song_id','listen_count']
song_df = pd.merge(othersongdata, songmetadata.drop_duplicates(['song_id']), on="song_id", how="left")
song_grouped = pd.DataFrame(song_df.groupby('song_id')['listen_count'].count())
song_grouped = pd.DataFrame(song_df.groupby('song_id')['listen_count'].count())
song_df.astype({'listen_count': 'int32'},{'song_id':'str'}).dtypes
song_df[song_df['song_id'] == givenSongID_From_FrontEnd]
songs_crosstab = pd.pivot_table(song_df, values = 'listen_count', index = 'user_id', columns = 'song_id')
songs_crosstab.head()
predictor_song_ratings = songs_crosstab[givenSongID_From_FrontEnd]
predictor_song_ratings[predictor_song_ratings>= 1]
similar_songs = songs_crosstab.corrwith(predictor_song_ratings)
corr_listened_song = pd.DataFrame(similar_songs, columns = ['pearsonR'])
corr_listened_song.dropna(inplace = True)
predictor_corr_summary =corr_listened_song.join(song_grouped['listen_count'])
predictor_corr_summary = predictor_corr_summary.sort_values('pearsonR', ascending = False)
final_recommended_songs = predictor_corr_summary[predictor_corr_summary.pearsonR < 0.9999]
final_recommended_songs.sort_values('pearsonR', ascending = False)
final_recommended_songs = final_recommended_songs.reset_index()
song_df_one = song_df.drop(['listen_count'], axis=1)
similar_songs = pd.merge(final_recommended_songs, song_df_one.drop_duplicates(["song_id"]), on="song_id", how="left")
similar_songs = similar_songs.sort_values('pearsonR', ascending = False)
similar_songs.head(50)

/Users/devarakondasantosh/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/devarakondasantosh/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,song_id,pearsonR,listen_count,user_id,title,release,artist_name,year
0,SOECOOL12AB0181A2F,0.999587,75,dd67a78d5f9d8140a0d83849441ca1807f7ea790,Fever,For Your Entertainment (Deluxe Version),Adam Lambert,2009
1,SOVMGAJ12AB0181580,0.999539,114,1b704d4cddabea8258bd93497fcb73eab32fa592,Stop This Song (Love Sick Melody) (Bonus Version),Riot!,Paramore,0
2,SOHQFVC12A58A7B234,0.999260,86,9b887e10a4711486085c4fae2d2599fc0d2c484d,Rock Wit U (Awww Baby),Def Jam 25_ Vol. 19 - For The Lover In You,Ashanti,2003
3,SOVBBUR12AB0181734,0.999097,162,732f88be38fae217f8ab7e24c20dd072436e3e40,There will be a happy meeting in Glory,Joseph Spence: The Complete Folkways Recording...,Joseph Spence,0
4,SOALPQQ12A6701F302,0.999055,200,5d5e0142e54c3bb7b69f548c2ee55066c90700eb,Put Your Head On My Shoulder (Album Version),Michael Bublé,Michael Bublé,2003
5,SOGFIHA12A8C13951F,0.998997,62,7012218281501a7500d921aabcf7a798dc7fdf38,Somewhere Out There,Forever More (Love Songs_ Hits & Duets),James Ingram;Linda Ronstadt,1993
6,SOQWVBM12AB0182B5F,0.998938,135,8f0e25d95d6e30f0993b9f57fca6c431d78f5a2d,Girl's Not Grey,Sing The Sorrow,AFI,2003
7,SOHYHMJ12A6D4F615E,0.998727,217,3fef4e71c70ae99ef8a4da00e431f9cdb2b1b729,The Best Of Times,20th Century Masters: The Millennium Collectio...,Styx,1981
8,SOOYOJU12A670201D3,0.998716,213,7c593e9db9a77a2dc8d3d975bb837422d1b0242a,Area Codes,Word Of Mouf,Ludacris / Nate Dogg,2001
9,SOOXREU12A8C132CEA,0.998270,134,5d5e0142e54c3bb7b69f548c2ee55066c90700eb,I'm Real,J.Lo,Jennifer Lopez featuring Ja Rule,2001


In [12]:
similar_songs

,song_id,pearsonR,listen_count,user_id,title,release,artist_name,year
0,SOECOOL12AB0181A2F,0.999587,75,dd67a78d5f9d8140a0d83849441ca1807f7ea790,Fever,For Your Entertainment (Deluxe Version),Adam Lambert,2009
1,SOVMGAJ12AB0181580,0.999539,114,1b704d4cddabea8258bd93497fcb73eab32fa592,Stop This Song (Love Sick Melody) (Bonus Version),Riot!,Paramore,0
2,SOHQFVC12A58A7B234,0.999260,86,9b887e10a4711486085c4fae2d2599fc0d2c484d,Rock Wit U (Awww Baby),Def Jam 25_ Vol. 19 - For The Lover In You,Ashanti,2003
3,SOVBBUR12AB0181734,0.999097,162,732f88be38fae217f8ab7e24c20dd072436e3e40,There will be a happy meeting in Glory,Joseph Spence: The Complete Folkways Recording...,Joseph Spence,0
4,SOALPQQ12A6701F302,0.999055,200,5d5e0142e54c3bb7b69f548c2ee55066c90700eb,Put Your Head On My Shoulder (Album Version),Michael Bublé,Michael Bublé,2003
...,...,...,...,...,...,...,...,...
3809,SOWBSGQ12A8C143163,-1.000000,197,cc9fc2eccf0d6fe78d1fb2b0c3ff924f54482169,Hits From The Bong,The Collection,Cypress Hill,1993
3810,SOSRERB12A8C139735,-1.000000,361,e376b24d90c39bdc1eb4b184b0e1374f2c112801,Lullaby,Sing-A-Longs & Lullabies For The Film Curious ...,Jack Johnson / Matt Costa,0
3811,SOYPOQC12A6701F459,-1.000000,88,043d81932e75d5749ed5758d6420506e7bc457a5,Why Do You Love Me (Clean version) (Re-mastere...,Absolute Garbage,Garbage,2007
3812,SOMRZNS12AF72AADC5,-1.000000,109,3187e658fd71be99d68d96f3a61a436c8a607365,No Regrets,I've Been Expecting You,Robbie Williams,1998


# Step 4: Recommender System 3 - User Based Recommender System

# Step 4: Making an Ensemble of All Recommenders

References

#Dataset:https://www.kaggle.com/mgmarques/million-song-recommendation-engines
#Links
#1) https://towardsdatascience.com/create-music-recommendation-system-using-python-ce5401317159
#2)https://github.com/llSourcell/recommender_live
#3) https://github.com/llSourcell/recommender_live/blob/master/Song%20Recommender_Python.ipynb
#4) https://towardsdatascience.com/how-to-build-a-simple-song-recommender-296fcbc8c85